In [15]:
import pandas as pd
import spacy
import sddk
import requests

In [2]:
nlp = spacy.load('la_core_web_lg')

In [3]:
s = sddk.cloudSession(provider="sciencedata.dk", shared_folder_name="TOME", owner="kase@zcu.cz")

connection with shared folder established with you as its ordinary user
endpoint variable has been configured to: https://sciencedata.dk/sharingout/kase%40zcu.cz/TOME/


In [6]:
dir_ids_list = s.list_directories("NOSCEMUS_FULL/")
dir_ids_list[:10]

['1031760',
 '1085290',
 '1285853',
 '1285854',
 '1285855',
 '1285856',
 '1365811',
 '1370560',
 '1378359',
 '1424044']

In [7]:
len(dir_ids_list)

1009

In [12]:
for id in dir_ids_list[:10]:
    for textfile in s.list_filenames("NOSCEMUS_FULL/" + id, "txt"):
        print(textfile)

Bacon,_Francis_-_Instauratio_magna__London_1620_pdf.txt
Linden,_Johannes_Antonides_van_der_-_Lindenius_renovatus__Nuremberg_1686_pdf.txt
de_Conde,_Ioannes_Baptista_-_Aphorismi_seu_axiomata_Hippocratis_versu_heroico_explicata__Brussels_1647_pdf.txt
van_Poort,_Henricus_-_Hippocratis_Aphorismi_metrica_praphrasi_translati__Utrecht_1657_pdf.txt
Hippocrates_&_Denisot,_Gérard_-_Hippocratis_Aphorismis_versibus_Graecis_et_Latini_expositi__Paris_1634_pdf.txt
Hippocrates_&_Berigardus,_Petrus_-_Hippocratis_Aphorismi_rhytmici__Udine_1645_pdf.txt
Acta_literaria_Sueciae__Vol__1__Uppsala_[1723–1725]_.txt
Quensel,_Conrad_&_Eurodius,_Hans_-_De_lumine_nocturno_Boreali__Lund_1726.txt
Celsius,_Anders_-_Observationes_de_lumine_Boreali__Nuremberg_1733.txt
Bacci,_Andrea_&_Gabelkover,_Wolfgang_-_De_monocerote_seu_unicornu__Stuttgart_1598_pdf.txt


In [13]:
ids_texts = []
for id in dir_ids_list[:10]:
    id_texts = []
    id_filenames = []
    for textfile in s.list_filenames("NOSCEMUS_FULL/" + id, "txt"):
        id_filenames.append(textfile)
        text = s.read_file("NOSCEMUS_FULL/" + id + "/" + textfile, "str")
        id_texts.append(text)
    ids_texts.append([id, id_filenames, id_texts])

In [17]:
pd.DataFrame(ids_texts, columns=["noscemus_ID", "filenames_list", "textcontent_list"])

,noscemus_ID,filenames_list,textcontent_list
0,1031760,"[Bacon,_Francis_-_Instauratio_magna__London_16...",[FRANCISCJ\nDE VERULAMIO /\nSummi Angliae\nCAN...
1,1085290,"[Linden,_Johannes_Antonides_van_der_-_Lindeniu...",[GEORG ABRAHAM MERCKLINI\nLINDENIUS RENOVATUS\...
2,1285853,"[de_Conde,_Ioannes_Baptista_-_Aphorismi_seu_ax...","[APHORISMI,\nSEU AXIOMATA\nHIPPOCRATIS,\nsepte..."
3,1285854,"[van_Poort,_Henricus_-_Hippocratis_Aphorismi_m...",[HIPPOCRATIS\nCOI\nAPHORISMI\nMetricâ\nPARAPHR...
4,1285855,"[Hippocrates_&_Denisot,_Gérard_-_Hippocratis_A...",[HIPPOCRATIS\nAPHORISMI\nVERSIBVS GRAECIS ET\n...
5,1285856,"[Hippocrates_&_Berigardus,_Petrus_-_Hippocrati...",[HIPPOCRATIS\nAphorismi Rhythmici\n\n\nA\n\n\n...
6,1365811,[Acta_literaria_Sueciae__Vol__1__Uppsala_[1723...,[ACTA\nLITERARIA\nSVECIAE\n\n\nUpsaliae public...
7,1370560,"[Quensel,_Conrad_&_Eurodius,_Hans_-_De_lumine_...",[Alb Haller D.\n1747.\nDono Jll. Roséni.\n\n\n...
8,1378359,"[Celsius,_Anders_-_Observationes_de_lumine_Bor...",[CCCXVI.\nOBSERVATIONES\nDE\n\n\nLUMEN\n\n\nBO...
9,1424044,"[Bacci,_Andrea_&_Gabelkover,_Wolfgang_-_De_mon...","[DE MONOCERO¬\nTE SEV\nVNICOR¬\nNV, EIVSQVE AD..."
